# Week 5 Exercise - Personal Knowledge Worker with RAG
### Author: Samuel Kalu, Team Euclid, Week 5

This notebook implements a RAG (Retrieval Augmented Generation) based Personal Knowledge Worker that can answer questions about your personal data.

Features:
- Document loading from multiple sources (Markdown files)
- Intelligent text chunking with overlap
- Vector embeddings using OpenAI/HuggingFace
- Chroma vector store for efficient retrieval
- t-SNE visualization (2D and 3D)
- Conversational RAG with memory
- Gradio chat interface
- Model switching support

## 1. Setup and Configuration

In [ ]:
# imports
import os
import glob
from pathlib import Path
from dotenv import load_dotenv
import gradio as gr
import numpy as np
import plotly.graph_objects as go
from sklearn.manifold import TSNE

# LangChain imports
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_huggingface import HuggingFaceEmbeddings

# Load environment variables
load_dotenv(override=True)

# API Keys
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

# Configuration
MODEL = "gpt-4.1-nano"
EMBEDDING_MODEL = "text-embedding-3-small"
DB_NAME = "personal_knowledge_db"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100
TOP_K_RESULTS = 5

# Knowledge base path
KNOWLEDGE_BASE_PATH = Path("knowledge_base")

## 2. Sample Knowledge Base Setup

In [ ]:
# Create sample knowledge base if it doesn't exist

def create_sample_knowledge_base():
    """Create a sample knowledge base with personal, projects, and learning data."""
    
    folders = ["personal", "projects", "learning"]
    
    for folder in folders:
        (KNOWLEDGE_BASE_PATH / folder).mkdir(parents=True, exist_ok=True)
    
    # Personal info
    personal_text = """
# Personal Profile

## About Me
Name: Samuel Kalu
Role: Software Engineer & AI Enthusiast
Location: Tech Hub City

## Background
I am a passionate software engineer with over 5 years of experience building scalable applications.
My journey started with web development and has evolved into specializing in AI and machine learning.

## Skills
- Programming Languages: Python, JavaScript, TypeScript, Go
- Frameworks: React, Node.js, FastAPI, LangChain
- AI/ML: LLMs, RAG systems, Vector Databases, Prompt Engineering
- Databases: PostgreSQL, MongoDB, Chroma, Pinecone

## Interests
I love exploring new technologies, contributing to open-source projects, and mentoring aspiring developers.
In my free time, I enjoy hiking, reading tech blogs, and experimenting with new AI tools.
"""
    
    # Projects
    projects_text = """
# Projects Portfolio

## AI-Powered Document Assistant
A RAG-based system that helps users query large document collections efficiently.
Tech Stack: Python, LangChain, Chroma, OpenAI API
Key Features: Semantic search, multi-document support, conversation history

## Real-time Analytics Dashboard
Built a scalable dashboard for visualizing business metrics in real-time.
Tech Stack: React, Node.js, PostgreSQL, Redis
Impact: Reduced reporting time by 80% for the operations team

## Code Review Automation Tool
An AI assistant that provides automated code reviews and suggestions.
Tech Stack: Python, GitHub API, LLM integration
Features: Pattern detection, best practices recommendations, security checks

## E-commerce Platform
Full-stack e-commerce solution with payment integration and inventory management.
Tech Stack: Next.js, Stripe, PostgreSQL, Docker
Scale: Handles 10,000+ daily active users
"""
    
    # Learning
    learning_text = """
# Learning Journey

## Current Focus Areas

### Large Language Models (LLMs)
Studying transformer architectures, attention mechanisms, and fine-tuning techniques.
Key Resources: HuggingFace courses, research papers, hands-on experimentation

### Retrieval Augmented Generation (RAG)
Learning about vector embeddings, semantic search, and retrieval strategies.
Practical Applications: Building knowledge bases, question-answering systems

### Vector Databases
Exploring Chroma, Pinecone, and Weaviate for efficient similarity search.
Use Cases: Recommendation systems, semantic caching, document retrieval

## Recent Learnings
- Advanced prompt engineering techniques (CoT, Few-shot learning)
- LangChain abstractions and chains
- Evaluation metrics for RAG systems
- Agentic AI and autonomous agents

## Goals
- Master advanced RAG architectures
- Contribute to open-source AI projects
- Build production-ready AI applications
"""
    
    # Write files
    (KNOWLEDGE_BASE_PATH / "personal" / "info.md").write_text(personal_text, encoding="utf-8")
    (KNOWLEDGE_BASE_PATH / "projects" / "portfolio.md").write_text(projects_text, encoding="utf-8")
    (KNOWLEDGE_BASE_PATH / "learning" / "journey.md").write_text(learning_text, encoding="utf-8")
    
    print(f"Sample knowledge base created at {KNOWLEDGE_BASE_PATH}")

# Create the sample data
create_sample_knowledge_base()

## 3. Document Ingestion and Vector Store

In [ ]:
# Load documents using LangChain loaders

def load_documents():
    """Load all markdown documents from the knowledge base."""
    
    folders = glob.glob(str(KNOWLEDGE_BASE_PATH / "*"))
    
    def add_metadata(doc, doc_type):
        """Add document type metadata."""
        doc.metadata["doc_type"] = doc_type
        doc.metadata["source"] = doc.metadata.get("source", "unknown")
        return doc
    
    text_loader_kwargs = {'encoding': 'utf-8'}
    documents = []
    
    for folder in folders:
        doc_type = Path(folder).name
        loader = DirectoryLoader(
            folder, 
            glob="**/*.md", 
            loader_cls=TextLoader, 
            loader_kwargs=text_loader_kwargs
        )
        folder_docs = loader.load()
        documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])
    
    return documents

# Load the documents
documents = load_documents()
print(f"Loaded {len(documents)} documents")
print(f"Document types: {set(doc.metadata['doc_type'] for doc in documents)}")

In [ ]:
# Split documents into chunks

text_splitter = CharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separator="\n"
)

chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Average chunk size: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f} characters")

In [ ]:
# Create vector store with Chroma

print("Initializing embeddings...")

# Use OpenAI embeddings (or switch to HuggingFace for free alternative)
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

# Delete existing collection if it exists
if Path(DB_NAME).exists():
    Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()
    print(f"Deleted existing vector store: {DB_NAME}")

# Create new vector store
print("Creating vector store...")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=DB_NAME
)

collection = vectorstore._collection
count = collection.count()

# Get sample embedding to determine dimensions
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)

print(f"Vectorstore created with {count:,} documents")
print(f"Embedding dimensions: {dimensions:,}")

## 4. Vector Store Visualization (t-SNE)

In [ ]:
# 2D t-SNE Visualization

def visualize_2d():
    """Create a 2D t-SNE visualization of the vector store."""
    
    result = collection.get(include=['embeddings', 'documents', 'metadatas'])
    vectors = np.array(result['embeddings'])
    documents = result['documents']
    metadatas = result['metadatas']
    doc_types = [metadata['doc_type'] for metadata in metadatas]
    
    # Color mapping for document types
    unique_types = list(set(doc_types))
    colors_map = {'personal': 'blue', 'projects': 'green', 'learning': 'red'}
    colors = [colors_map.get(t, 'gray') for t in doc_types]
    
    # t-SNE requires at least 3 samples
    n = vectors.shape[0]
    if n < 3:
        print(f"t-SNE needs at least 3 samples, got {n}")
        return None
    
    # Calculate perplexity (should be less than number of samples)
    perplexity = max(5.0, min(30.0, (n - 1) / 3.0))
    
    print(f"Running t-SNE with perplexity={perplexity:.1f}...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity, n_iter=1000)
    reduced_vectors = tsne.fit_transform(vectors)
    
    fig = go.Figure(data=[go.Scatter(
        x=reduced_vectors[:, 0],
        y=reduced_vectors[:, 1],
        mode='markers',
        marker=dict(size=6, color=colors, opacity=0.7, line=dict(width=1, color='white')),
        text=[f"Type: {t}<br>Text: {d[:150]}..." for t, d in zip(doc_types, documents)],
        hoverinfo='text',
        name='Chunks'
    )])
    
    fig.update_layout(
        title='2D Vector Store Visualization (t-SNE)',
        xaxis_title='t-SNE Dimension 1',
        yaxis_title='t-SNE Dimension 2',
        width=900,
        height=700,
        hovermode='closest',
        template='plotly_white',
        legend=dict(x=0, y=1)
    )
    
    return fig

# Show 2D visualization
fig_2d = visualize_2d()
if fig_2d:
    fig_2d.show()

In [ ]:
# 3D t-SNE Visualization

def visualize_3d():
    """Create a 3D t-SNE visualization of the vector store."""
    
    result = collection.get(include=['embeddings', 'documents', 'metadatas'])
    vectors = np.array(result['embeddings'])
    documents = result['documents']
    metadatas = result['metadatas']
    doc_types = [metadata['doc_type'] for metadata in metadatas]
    
    unique_types = list(set(doc_types))
    colors_map = {'personal': 'blue', 'projects': 'green', 'learning': 'red'}
    colors = [colors_map.get(t, 'gray') for t in doc_types]
    
    n = vectors.shape[0]
    if n < 3:
        print(f"t-SNE needs at least 3 samples, got {n}")
        return None
    
    perplexity = max(5.0, min(30.0, (n - 1) / 3.0))
    
    print(f"Running 3D t-SNE with perplexity={perplexity:.1f}...")
    tsne = TSNE(n_components=3, random_state=42, perplexity=perplexity, n_iter=1000)
    reduced_vectors = tsne.fit_transform(vectors)
    
    fig = go.Figure(data=[go.Scatter3d(
        x=reduced_vectors[:, 0],
        y=reduced_vectors[:, 1],
        z=reduced_vectors[:, 2],
        mode='markers',
        marker=dict(size=5, color=colors, opacity=0.8),
        text=[f"Type: {t}<br>Text: {d[:150]}..." for t, d in zip(doc_types, documents)],
        hoverinfo='text'
    )])
    
    fig.update_layout(
        title='3D Vector Store Visualization (t-SNE)',
        scene=dict(
            xaxis_title='t-SNE Dimension 1',
            yaxis_title='t-SNE Dimension 2',
            zaxis_title='t-SNE Dimension 3'
        ),
        width=1000,
        height=800,
        margin=dict(r=10, b=10, l=10, t=50)
    )
    
    return fig

# Show 3D visualization
fig_3d = visualize_3d()
if fig_3d:
    fig_3d.show()

## 5. RAG Chain Setup

In [ ]:
# Setup the RAG chain with conversation memory

print("Setting up RAG chain...")

# Initialize the LLM
llm = ChatOpenAI(
    temperature=0.7,
    model_name=MODEL,
    streaming=True
)

# Create retriever with custom search parameters
retriever = vectorstore.as_retriever(
    search_kwargs={"k": TOP_K_RESULTS}
)

# Setup conversation memory
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True,
    output_key='answer'
)

# Create the conversational retrieval chain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

print("RAG chain initialized successfully!")

# Test the chain
def chat(question: str, history: list) -> str:
    """Process a question and return an answer."""
    try:
        result = conversation_chain.invoke({"question": question})
        answer = result["answer"]
        
        # Add source information
        sources = set()
        for doc in result.get("source_documents", []):
            doc_type = doc.metadata.get("doc_type", "unknown")
            sources.add(doc_type)
        
        if sources:
            answer += f"\n\n_Sources: {', '.join(sources)}_"
        
        return answer
    except Exception as e:
        return f"Error: {str(e)}"

# Quick test
test_question = "What is my background?"
print(f"\nTest question: {test_question}")
test_answer = chat(test_question, [])
print(f"Test answer: {test_answer[:200]}...")

## 6. Gradio Chat Interface

In [ ]:
# Create Gradio UI

def create_gradio_interface():
    """Create the Gradio chat interface."""
    
    with gr.Blocks(
        theme=gr.themes.Soft(
            primary_hue=gr.themes.colors.blue,
            spacing_size=gr.themes.sizes.spacing_md
        ),
        title="Personal Knowledge Worker - Samuel Kalu, Team Euclid"
    ) as ui:
        
        gr.Markdown(
            """
            # Personal Knowledge Worker
            ### By Samuel Kalu, Team Euclid - Week 5
            
            Chat with your personal knowledge base powered by RAG (Retrieval Augmented Generation).
            Ask questions about your profile, projects, and learning journey.
            """
        )
        
        # Chat interface
        chat_interface = gr.ChatInterface(
            fn=chat,
            type="messages",
            title="Knowledge Base Chat",
            description="Ask anything about your personal data",
            examples=[
                ["What is my background?"],
                ["Tell me about my projects"],
                ["What am I currently learning?"],
                ["What are my main skills?"]
            ],
            retry_btn=None,
            undo_btn=None,
            clear_btn="Clear Chat"
        )
        
        # Additional info
        gr.Markdown(
            """
            ---
            **Features:**
            - Semantic search across all your documents
            - Conversation history for context-aware responses
            - Source attribution for answers
            - Powered by OpenAI embeddings and GPT-4.1-nano
            
            **Tips:**
            - Ask specific questions for better answers
            - Follow-up questions work thanks to conversation memory
            - Check the source documents for more details
            """
        )
    
    return ui

print("Gradio interface configured!")

## 7. Launch the Application

In [ ]:
# Launch the Gradio UI

if __name__ == "__main__":
    ui = create_gradio_interface()
    print("\nLaunching Personal Knowledge Worker...")
    print("Open the URL shown below in your browser to start chatting.")
    print("Press Ctrl+C to stop the server.\n")
    ui.launch(share=False, inbrowser=True)

## Summary

This Week 5 exercise solution demonstrates a complete RAG pipeline:

### Key Components:
1. **Document Loading**: Uses LangChain DirectoryLoader to load markdown files
2. **Text Chunking**: CharacterTextSplitter with configurable chunk size and overlap
3. **Vector Embeddings**: OpenAI embeddings (text-embedding-3-small)
4. **Vector Store**: Chroma for persistent storage and retrieval
5. **Visualization**: t-SNE for 2D/3D visualization of document clusters
6. **RAG Chain**: ConversationalRetrievalChain with memory
7. **Chat Interface**: Gradio with conversation history

### Features Implemented:
- Multi-document type support (personal, projects, learning)
- Conversation memory for context-aware responses
- Source attribution in answers
- Interactive 2D and 3D visualizations
- Clean, modern Gradio UI
- Error handling and graceful degradation

### Business Applications:
- Employee knowledge bases
- Company documentation Q&A
- Personal note-taking assistants
- Customer support automation
- Research paper search engines

### Author: Samuel Kalu, Team Euclid, Week 5